In [4]:
# Import the libraries
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

In [5]:
# read the files
# file path
providers_path = "providers_data.csv"
receivers_path = "receivers_data.csv"
food_path      = "food_listings_data.csv"
claims_path    = "claims_data.csv"

# Read the files using 'pd.read_csv'
providers  = pd.read_csv(providers_path)
receivers  = pd.read_csv(receivers_path)
food       = pd.read_csv(food_path)
claims     = pd.read_csv(claims_path)

# Peek
for name, df in [("providers", providers), ("receivers", receivers), ("food", food), ("claims", claims)]:
    print(f"\n=== {name.upper()} ===")
    print(df.shape)
    print(df.head(3))



=== PROVIDERS ===
(1000, 6)
   Provider_ID                         Name           Type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson and Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   

                                             Address            City  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   

                Contact  
0       +1-600-220-0480  
1  +1-925-283-8901x6297  
2      001-517-295-2206  

=== RECEIVERS ===
(1000, 5)
   Receiver_ID          Name        Type               City           Contact
0            1  Donald Gomez     Shelter     Port Carlburgh     (955)922-5295
1            2  Laurie Ramos  Individual         Lewisburgh      761.042.1570
2            3  Ashley Mckee         NGO  South Randalltown  691-0

In [6]:
# Clean and standardise the datatype
# Strip spaces in object columns
def strip_obj_cols(df):
    obj_cols = df.select_dtypes(include="object").columns
    df[obj_cols] = df[obj_cols].apply(lambda s: s.str.strip())
    return df

providers = strip_obj_cols(providers)
receivers = strip_obj_cols(receivers)
food = strip_obj_cols(food)
claims = strip_obj_cols(claims)

# Parse dates
# food['Expiry_Date'] is in string so now we will convert it to date (yyyy-mm-dd)
food['Expiry_Date'] = pd.to_datetime(food['Expiry_Date'], errors='coerce').dt.date

# claims['Timestamp'] is in string so we will also convert it to proper datetime
claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], errors='coerce')

# Normalize Status values
valid_status = {"Pending","Completed","Cancelled"}
claims['Status'] = claims['Status'].str.title().where(claims['Status'].str.title().isin(valid_status), other="Pending")

# Quick null checks
for name, df in [("providers", providers), ("receivers", receivers), ("food", food), ("claims", claims)]:
    print(f"{name}: nulls\n", df.isna().sum())


providers: nulls
 Provider_ID    0
Name           0
Type           0
Address        0
City           0
Contact        0
dtype: int64
receivers: nulls
 Receiver_ID    0
Name           0
Type           0
City           0
Contact        0
dtype: int64
food: nulls
 Food_ID          0
Food_Name        0
Quantity         0
Expiry_Date      0
Provider_ID      0
Provider_Type    0
Location         0
Food_Type        0
Meal_Type        0
dtype: int64
claims: nulls
 Claim_ID       0
Food_ID        0
Receiver_ID    0
Status         0
Timestamp      0
dtype: int64


In [7]:
# create sqlite database
db_path = "food.db"
conn = sqlite3.connect(db_path)
conn.execute("PRAGMA foreign_keys = ON;")  # enforce FK
conn.execute("PRAGMA journal_mode = WAL;") # safer writes

# Write to staging (drops if exist)
providers.to_sql("stg_providers", conn, if_exists="replace", index=False)
receivers.to_sql("stg_receivers", conn, if_exists="replace", index=False)
food.to_sql("stg_food", conn, if_exists="replace", index=False)
claims.to_sql("stg_claims", conn, if_exists="replace", index=False)

print("Staging tables created.")


Staging tables created.


In [8]:
# create final table with constraints
cur = conn.cursor()

# Drop if re-running
cur.executescript("""
DROP TABLE IF EXISTS Claims;
DROP TABLE IF EXISTS Food_Listings;
DROP TABLE IF EXISTS Receivers;
DROP TABLE IF EXISTS Providers;
""")

# Create final tables
cur.executescript("""
CREATE TABLE Providers (
    Provider_ID   INTEGER PRIMARY KEY,
    Name          TEXT NOT NULL,
    Type          TEXT,
    Address       TEXT,
    City          TEXT,
    Contact       TEXT
);

CREATE TABLE Receivers (
    Receiver_ID   INTEGER PRIMARY KEY,
    Name          TEXT NOT NULL,
    Type          TEXT,
    City          TEXT,
    Contact       TEXT
);

CREATE TABLE Food_Listings (
    Food_ID       INTEGER PRIMARY KEY,
    Food_Name     TEXT NOT NULL,
    Quantity      INTEGER CHECK(Quantity >= 0),
    Expiry_Date   DATE,
    Provider_ID   INTEGER NOT NULL,
    Provider_Type TEXT,
    Location      TEXT,
    Food_Type     TEXT,
    Meal_Type     TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES Providers(Provider_ID) ON DELETE CASCADE
);

CREATE TABLE Claims (
    Claim_ID      INTEGER PRIMARY KEY,
    Food_ID       INTEGER NOT NULL,
    Receiver_ID   INTEGER NOT NULL,
    Status        TEXT CHECK(Status IN ('Pending','Completed','Cancelled')) NOT NULL,
    Timestamp     DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES Food_Listings(Food_ID) ON DELETE CASCADE,
    FOREIGN KEY (Receiver_ID) REFERENCES Receivers(Receiver_ID) ON DELETE CASCADE
);
""")
conn.commit()
print("Final tables created.")


Final tables created.


In [9]:
# Load from staging
cur.executescript("""
INSERT OR IGNORE INTO Providers (Provider_ID, Name, Type, Address, City, Contact)
SELECT DISTINCT Provider_ID, Name, Type, Address, City, Contact
FROM stg_providers;

INSERT OR IGNORE INTO Receivers (Receiver_ID, Name, Type, City, Contact)
SELECT DISTINCT Receiver_ID, Name, Type, City, Contact
FROM stg_receivers;

INSERT OR IGNORE INTO Food_Listings (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
SELECT DISTINCT Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type
FROM stg_food;

INSERT OR IGNORE INTO Claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
SELECT DISTINCT Claim_ID, Food_ID, Receiver_ID, Status, Timestamp
FROM stg_claims;
""")
conn.commit()

# Indexes for speed
cur.executescript("""
CREATE INDEX IF NOT EXISTS idx_food_provider    ON Food_Listings(Provider_ID);
CREATE INDEX IF NOT EXISTS idx_food_location    ON Food_Listings(Location);
CREATE INDEX IF NOT EXISTS idx_claims_food      ON Claims(Food_ID);
CREATE INDEX IF NOT EXISTS idx_claims_receiver  ON Claims(Receiver_ID);
CREATE INDEX IF NOT EXISTS idx_claims_status    ON Claims(Status);
""")
conn.commit()

for t in ["Providers","Receivers","Food_Listings","Claims"]:
    print(t, pd.read_sql_query(f"SELECT COUNT(*) AS n FROM {t}", conn))


Providers       n
0  1000
Receivers       n
0  1000
Food_Listings       n
0  1000
Claims       n
0  1000


In [10]:
# helper to run sql and see results
def q(sql, params=()):
    return pd.read_sql_query(sql, conn, params=params)

# quick smoke test
q("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

,name
0,Claims
1,Food_Listings
2,Providers
3,Receivers
4,stg_claims
5,stg_food
6,stg_providers
7,stg_receivers


In [11]:
# Exploratory Data Analysis
# 1. Providers and reciever count per city
print("Providers per city")
display(q("""
SELECT City, COUNT(*) AS providers
FROM Providers
GROUP BY City
ORDER BY providers DESC;
"""))

print("Receivers per city")
display(q("""
SELECT City, COUNT(*) AS receivers
FROM Receivers
GROUP BY City
ORDER BY receivers DESC;
"""))

Providers per city


,City,providers
0,South Christopherborough,3
1,New Carol,3
2,Williamview,2
3,West Lauraborough,2
4,West Christopher,2
...,...,...
958,Alexanderchester,1
959,Aguirreville,1
960,Adamsville,1
961,Adamsview,1


Receivers per city


,City,receivers
0,New Christopher,3
1,Williamsfort,2
2,West Richard,2
3,West James,2
4,Stevenmouth,2
...,...,...
961,Alexanderbury,1
962,Aguilarstad,1
963,Aguilarbury,1
964,Adamland,1


In [12]:
# 2. Which provider contribute the most food (by total Quantity listed)
q("""
SELECT Provider_Type, SUM(Quantity) AS total_quantity
FROM Food_Listings
GROUP BY Provider_Type
ORDER BY total_quantity DESC;
""")

,Provider_Type,total_quantity
0,Restaurant,6923
1,Supermarket,6696
2,Catering Service,6116
3,Grocery Store,6059


In [13]:
# 3. Contact info of a provider in specific Citycity
city = "Valentineside"
q("""
SELECT Name, Type, Contact, Address
FROM Providers
WHERE City = ?
ORDER BY Name;
""", (city,))


,Name,Type,Contact,Address
0,Coleman-Farley,Grocery Store,193.714.6577,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978"


In [14]:
# 4. Recievers who claimed the most (By number of claims)
q("""
SELECT r.Receiver_ID, r.Name, r.City, COUNT(*) AS claims_count
FROM Claims c
JOIN Receivers r ON r.Receiver_ID = c.Receiver_ID
GROUP BY r.Receiver_ID, r.Name, r.City
ORDER BY claims_count DESC
LIMIT 10;
""")

,Receiver_ID,Name,City,claims_count
0,276,Scott Hunter,Greenton,5
1,371,William Frederick,Port Dean,5
2,742,Matthew Webb,West David,5
3,800,Anthony Garcia,Brownbury,5
4,90,Alvin West,Kellybury,4
5,144,Jennifer Nelson,North Paul,4
6,301,Kristina Simpson,Melissaport,4
7,324,Kristine Martin,Thompsonhaven,4
8,410,Betty Reid,North Abigail,4
9,493,Chelsea Powell,Lake Rachelburgh,4


In [15]:
# 5. Recivers who claimed most (By total quantity)
q("""
SELECT r.Receiver_ID, r.Name, r.City,
       SUM(f.Quantity) AS total_quantity_claimed
FROM Claims c
JOIN Receivers r ON r.Receiver_ID = c.Receiver_ID
JOIN Food_Listings f ON f.Food_ID = c.Food_ID
GROUP BY r.Receiver_ID, r.Name, r.City
ORDER BY total_quantity_claimed DESC
LIMIT 10;
""")

,Receiver_ID,Name,City,total_quantity_claimed
0,742,Matthew Webb,West David,191
1,901,Donald Caldwell,Dawsonberg,174
2,800,Anthony Garcia,Brownbury,162
3,276,Scott Hunter,Greenton,157
4,144,Jennifer Nelson,North Paul,139
5,493,Chelsea Powell,Lake Rachelburgh,134
6,473,Justin Powell,Port Sarah,126
7,301,Kristina Simpson,Melissaport,124
8,745,Jose Vance,Lake Heatherberg,123
9,306,Donna Williams,Calebview,122


In [16]:
# 6. Total quantity of food available from all providers
q("""SELECT
        SUM(Quantity) AS total_quantity_available
        FROM Food_Listings;
        """)

,total_quantity_available
0,25794


In [17]:
# 7. City with highest number of food listings
q("""
SELECT Location AS City, COUNT(*) AS listings
FROM Food_Listings
GROUP BY Location
ORDER BY listings DESC
LIMIT 10;
""")

,City,listings
0,New Carol,6
1,South Kathryn,6
2,East Angela,5
3,Jimmyberg,5
4,Perezport,5
5,Davidville,4
6,East Heatherport,4
7,East Stephanie,4
8,Frederickside,4
9,Jonathanstad,4


In [18]:
# 8. Most commonly avalaible food type
q("""
SELECT Food_Type, COUNT(*) AS items
FROM Food_Listings
GROUP BY Food_Type
ORDER BY items DESC;
""")

,Food_Type,items
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


In [19]:
# 9. Claims per food item
q("""
SELECT c.Food_ID, f.Food_Name, COUNT(*) AS claims_count
FROM Claims c
JOIN Food_Listings f ON f.Food_ID = c.Food_ID
GROUP BY c.Food_ID, f.Food_Name
ORDER BY claims_count DESC
LIMIT 15;
""")

,Food_ID,Food_Name,claims_count
0,463,Soup,5
1,486,Chicken,5
2,548,Fish,5
3,35,Rice,4
4,92,Chicken,4
5,190,Salad,4
6,217,Soup,4
7,251,Dairy,4
8,520,Fish,4
9,674,Vegetables,4


In [20]:
# 10. Providers with highest number of successfull(completed) claims
q("""
SELECT p.Provider_ID, p.Name, COUNT(*) AS completed_claims
FROM Claims c
JOIN Food_Listings f ON f.Food_ID = c.Food_ID
JOIN Providers p ON p.Provider_ID = f.Provider_ID
WHERE c.Status = 'Completed'
GROUP BY p.Provider_ID, p.Name
ORDER BY completed_claims DESC
LIMIT 10;
""")

,Provider_ID,Name,completed_claims
0,709,Barry Group,5
1,241,"Harper, Blake and Alexander",4
2,596,"Barnes, Castro and Curtis",4
3,752,Butler-Richardson,4
4,967,Miller Inc,4
5,161,Campbell LLC,3
6,262,Bradford-Martinez,3
7,315,Nelson LLC,3
8,384,Rogers-Warren,3
9,538,Beck Inc,3


In [41]:
# 11. % of claims by status
q("""
WITH t AS (
    SELECT Status, COUNT(*) AS cnt
    FROM Claims
    GROUP BY Status
), total AS (
    SELECT SUM(cnt) AS total_cnt FROM t
)
SELECT t.Status, t.cnt,
       ROUND(100.0 * t.cnt / total.total_cnt, 2) AS pct
FROM t, total
ORDER BY pct DESC;
""")

,Status,cnt,pct
0,Completed,339,33.9
1,Cancelled,336,33.6
2,Pending,325,32.5


In [21]:
# 12. Average quantity claimed per reciever
q("""
SELECT r.Receiver_ID, r.Name,
       ROUND(AVG(f.Quantity), 2) AS avg_qty_per_claim
FROM Claims c
JOIN Receivers r ON r.Receiver_ID = c.Receiver_ID
JOIN Food_Listings f ON f.Food_ID = c.Food_ID
GROUP BY r.Receiver_ID, r.Name
ORDER BY avg_qty_per_claim DESC
LIMIT 10;
""")

,Receiver_ID,Name,avg_qty_per_claim
0,99,Nancy Silva,50.0
1,282,Lisa Pitts,50.0
2,429,Daniel Williams,50.0
3,567,Peggy Knight,50.0
4,616,Christopher Wright,50.0
5,739,Nancy Jones,50.0
6,982,Thomas Villanueva,50.0
7,37,Angel Adams,49.0
8,38,Alexander Mullins,49.0
9,69,Scott Brown,49.0


In [22]:
# 13. Meal type claimed the most (by count)
q("""
SELECT f.Meal_Type, COUNT(*) AS claim_count
FROM Claims c
JOIN Food_Listings f ON f.Food_ID = c.Food_ID
GROUP BY f.Meal_Type
ORDER BY claim_count DESC;
""")

,Meal_Type,claim_count
0,Breakfast,278
1,Lunch,250
2,Snacks,240
3,Dinner,232


In [23]:
# 14. Total quantity donated per provider
q("""
SELECT p.Provider_ID, p.Name, SUM(f.Quantity) AS total_qty_donated
FROM Food_Listings f
JOIN Providers p ON p.Provider_ID = f.Provider_ID
GROUP BY p.Provider_ID, p.Name
ORDER BY total_qty_donated DESC
LIMIT 10;
""")

,Provider_ID,Name,total_qty_donated
0,709,Barry Group,179
1,306,"Evans, Wright and Mitchell",158
2,655,Smith Group,150
3,315,Nelson LLC,142
4,678,Ruiz-Oneal,140
5,499,Blankenship-Lewis,124
6,41,Kelly-Ware,123
7,161,Campbell LLC,123
8,262,Bradford-Martinez,121
9,146,Hampton-Lee,116


In [24]:
# 15. Monthly claims trend
q("""
SELECT strftime('%Y-%m', Timestamp) AS ym, COUNT(*) AS claims
FROM Claims
GROUP BY ym
ORDER BY ym;
""")

,ym,claims
0,2025-03,1000


In [25]:
# 16. Top cities by completed claims
q("""
SELECT f.Location AS City, COUNT(*) AS completed_claims
FROM Claims c
JOIN Food_Listings f ON f.Food_ID = c.Food_ID
WHERE c.Status = 'Completed'
GROUP BY f.Location
ORDER BY completed_claims DESC
LIMIT 10;
""")

,City,completed_claims
0,South Kathryn,5
1,Zimmermanville,4
2,New Carol,4
3,East Heatherport,4
4,Devinmouth,4
5,Coleburgh,4
6,Shortfurt,3
7,Phillipsfort,3
8,Perezport,3
9,Patrickfort,3


In [26]:
# CRUD Operations
# 1. Create- add a new provider
cur = conn.cursor()
cur.execute("""
INSERT INTO Providers (Provider_ID, Name, Type, Address, City, Contact)
VALUES (?, ?, ?, ?, ?, ?);
""", (99999, "Demo Cafe", "Restaurant", "123 Demo Street", "DemoCity", "+1-000-000-0000"))
conn.commit()

q("SELECT * FROM Providers WHERE Provider_ID = 99999;")

,Provider_ID,Name,Type,Address,City,Contact
0,99999,Demo Cafe,Restaurant,123 Demo Street,DemoCity,+1-000-000-0000


In [27]:
# 2. Read - Fetch the provider in a city
q("SELECT Name, Type, City, Contact FROM Providers WHERE City = ?;", ("DemoCity",))

,Name,Type,City,Contact
0,Demo Cafe,Restaurant,DemoCity,+1-000-000-0000


In [28]:
# Update - Change the food listings quantity
# Pick any existing Food_ID first
fid = q("SELECT Food_ID FROM Food_Listings LIMIT 1;")['Food_ID'].iloc[0]
print("Updating Food_ID:", fid)

cur.execute("UPDATE Food_Listings SET Quantity = Quantity + 5 WHERE Food_ID = ?;", (fid,))
conn.commit()

q("SELECT Food_ID, Food_Name, Quantity FROM Food_Listings WHERE Food_ID = ?;", (fid,))

Updating Food_ID: 23


,Food_ID,Food_Name,Quantity


In [29]:
# create app.py
%%writefile app.py
# app.py - Streamlit app for Local Food Wastage Management System
import sqlite3
import pandas as pd
import streamlit as st

DB = "food.db"

@st.cache_data
def load_table(sql, params=()):
    with sqlite3.connect(DB) as conn:
        return pd.read_sql_query(sql, conn, params=params)

def run_write(sql, params=()):
    with sqlite3.connect(DB) as conn:
        cur = conn.cursor()
        cur.execute("PRAGMA foreign_keys = ON;")
        cur.execute(sql, params)
        conn.commit()

st.set_page_config(page_title="Local Food Wastage Management", layout="wide")
st.title("Local Food Wastage Management System")

tab1, tab2, tab3, tab4 = st.tabs(["Browse & Filter", "SQL Insights", "Providers (CRUD)", "Claims (CRUD)"])

with tab1:
    st.subheader("Filter food listings")
    cities = ["All"] + sorted(load_table("SELECT DISTINCT Location AS City FROM Food_Listings;")["City"].dropna().tolist())
    provider_types = ["All"] + sorted(load_table("SELECT DISTINCT Provider_Type FROM Food_Listings;")["Provider_Type"].dropna().tolist())
    food_types = ["All"] + sorted(load_table("SELECT DISTINCT Food_Type FROM Food_Listings;")["Food_Type"].dropna().tolist())
    meal_types = ["All"] + sorted(load_table("SELECT DISTINCT Meal_Type FROM Food_Listings;")["Meal_Type"].dropna().tolist())

    c_city = st.selectbox("City", cities)
    c_pt   = st.selectbox("Provider Type", provider_types)
    c_ft   = st.selectbox("Food Type", food_types)
    c_mt   = st.selectbox("Meal Type", meal_types)

    sql = "SELECT * FROM Food_Listings WHERE 1=1"
    params = []
    if c_city != "All":
        sql += " AND Location = ?"; params.append(c_city)
    if c_pt != "All":
        sql += " AND Provider_Type = ?"; params.append(c_pt)
    if c_ft != "All":
        sql += " AND Food_Type = ?"; params.append(c_ft)
    if c_mt != "All":
        sql += " AND Meal_Type = ?"; params.append(c_mt)

    st.dataframe(load_table(sql, params))

with tab2:
    st.subheader("Key SQL insights")
    st.markdown("**Providers per City**")
    st.dataframe(load_table("""
        SELECT City, COUNT(*) AS providers
        FROM Providers GROUP BY City ORDER BY providers DESC;
    """))

    st.markdown("**Completed Claims by Provider**")
    st.dataframe(load_table("""
        SELECT p.Name, COUNT(*) AS completed_claims
        FROM Claims c
        JOIN Food_Listings f ON f.Food_ID = c.Food_ID
        JOIN Providers p ON p.Provider_ID = f.Provider_ID
        WHERE c.Status='Completed'
        GROUP BY p.Name ORDER BY completed_claims DESC LIMIT 15;
    """))

    st.markdown("**Status Mix (%)**")
    st.dataframe(load_table("""
        WITH t AS (
            SELECT Status, COUNT(*) AS cnt FROM Claims GROUP BY Status
        ), total AS (
            SELECT SUM(cnt) AS total_cnt FROM t
        )
        SELECT t.Status, t.cnt,
               ROUND(100.0 * t.cnt / total.total_cnt, 2) AS pct
        FROM t, total
        ORDER BY pct DESC;
    """))

with tab3:
    st.subheader("Manage Providers")
    st.markdown("### Add New")
    with st.form("add_provider"):
        pid = st.number_input("Provider_ID", step=1)
        name = st.text_input("Name")
        typ = st.text_input("Type")
        addr = st.text_input("Address")
        city = st.text_input("City")
        contact = st.text_input("Contact")
        submitted = st.form_submit_button("Insert")
        if submitted:
            run_write("""INSERT OR REPLACE INTO Providers
                         (Provider_ID, Name, Type, Address, City, Contact)
                         VALUES (?, ?, ?, ?, ?, ?);""",
                      (pid, name, typ, addr, city, contact))
            st.success("Inserted/Updated.")
    st.markdown("### Current Providers")
    st.dataframe(load_table("SELECT * FROM Providers ORDER BY Provider_ID;"))

with tab4:
    st.subheader("Manage Claims")
    st.markdown("### Add Claim")
    with st.form("add_claim"):
        cid = st.number_input("Claim_ID", step=1)
        fid = st.number_input("Food_ID", step=1)
        rid = st.number_input("Receiver_ID", step=1)
        status = st.selectbox("Status", ["Pending","Completed","Cancelled"])
        submitted = st.form_submit_button("Insert")
        if submitted:
            run_write("""INSERT OR REPLACE INTO Claims
                         (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
                         VALUES (?, ?, ?, ?, datetime('now'));""",
                      (cid, fid, rid, status))
            st.success("Claim inserted/updated.")

    st.markdown("### Existing Claims")
    st.dataframe(load_table("SELECT * FROM Claims ORDER BY Timestamp DESC LIMIT 100;"))


Writing app.py


In [30]:
# Deployment using streamlit and pyngrok
!pip install streamlit pyngrok pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [40]:
!ngrok config add-authtoken 2z0RR8QG1qq5BX9oqijdzFXnW2P_5Wvt3GBDKYTmc4Hwgmjqo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [41]:
from pyngrok import ngrok
import time

# Kill any previous tunnels
ngrok.kill()

# Start Streamlit in background
get_ipython().system_raw("streamlit run app.py --server.port 8501 &")

# Wait a few seconds
time.sleep(5)

# Create tunnel
public_url = ngrok.connect(8501)
print("🚀 Your Streamlit app is live at:", public_url)

🚀 Your Streamlit app is live at: NgrokTunnel: "https://f2d8711c03d2.ngrok-free.app" -> "http://localhost:8501"
